<a href="https://colab.research.google.com/github/SWALIHSHA/AKNM-CLG/blob/main/Torrent_To_Google_Drive_Downloader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Torrent To Google Drive Downloader

**Important Note:** To get more disk space:
> Go to Runtime -> Change Runtime and give GPU as the Hardware Accelerator.  You will get around 384GB to download any torrent you want.

Add `%load_ext cuml.accel` before importing sklearn to speed up operations using GPU


In [5]:
# Step 1: Remove the broken apt version
!apt-get remove -y python3-libtorrent

# Step 2: Install a working precompiled pip wheel for libtorrent
!pip install libtorrent==2.0.11


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Package 'python3-libtorrent' is not installed, so not removed
The following package was automatically installed and is no longer required:
  libtorrent-rasterbar2.0
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 51.7 MB/s eta 0:00:00


In [6]:
# Step 3: Import and use libtorrent
import libtorrent as lt

ses = lt.session()
ses.listen_on(6881, 6891)
downloads = []



/tmp/ipython-input-6-3522283334.py:5: DeprecationWarning: listen_on() is deprecated
  ses.listen_on(6881, 6891)


### Install libtorrent and Initialize Session

### Mount Google Drive
To stream files we need to mount Google Drive.

In [7]:
from google.colab import drive

drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Add From Torrent File
You can run this cell to add more files as many times as you want

In [ ]:
from google.colab import files

source = files.upload()
params = {
    "save_path": "/content/drive/My Drive/Torrent",
    "ti": lt.torrent_info(list(source.keys())[0]),
}
downloads.append(ses.add_torrent(params))

### Add From Magnet Link
You can run this cell to add more files as many times as you want

In [8]:
params = {"save_path": "/content/drive/My Drive/Torrent"}

while True:
    magnet_link = input("Enter Magnet Link Or Type Exit: ")
    if magnet_link.lower() == "exit":
        break
    downloads.append(
        lt.add_magnet_uri(ses, magnet_link, params)
    )


Enter Magnet Link Or Type Exit: magnet:?xt=urn:btih:dcd4caf265dab5382616be3ae7cf7ed80e951f97&dn=www.1TamilMV.boo%20-%20Kerala%20Crime%20Files%20%282025%29%20S02%20EP%2801-06%29%20WEB-DL%20-%201080p%20-%20AVC%20-%20%5BTam%20%2B%20Mal%20%2B%20Tel%20%2B%20Hin%20%2B%20Kan%5D%20-%205.8GB%20-%20ESub&xl=6324818321&tr=udp%3A%2F%2Ftracker.opentrackr.org%3A1337%2Fannounce&tr=udp%3A%2F%2Fopen.demonii.com%3A1337%2Fannounce&tr=udp%3A%2F%2Fopen.tracker.cl%3A1337%2Fannounce&tr=udp%3A%2F%2Fopen.stealth.si%3A80%2Fannounce&tr=udp%3A%2F%2Ftracker.torrent.eu.org%3A451%2Fannounce&tr=udp%3A%2F%2Fexplodie.org%3A6969%2Fannounce&tr=udp%3A%2F%2Fexodus.desync.com%3A6969%2Fannounce&tr=udp%3A%2F%2Ftracker.ololosh.space%3A6969%2Fannounce&tr=udp%3A%2F%2Ftracker.dump.cl%3A6969%2Fannounce&tr=udp%3A%2F%2Ftracker.bittor.pw%3A1337%2Fannounce&tr=udp%3A%2F%2Ftracker-udp.gbitt.info%3A80%2Fannounce&tr=udp%3A%2F%2Fopentracker.io%3A6969%2Fannounce&tr=udp%3A%2F%2Fisk.richardsw.club%3A6969%2Fannounce&tr=udp%3A%2F%2Fdiscord.heiha

/tmp/ipython-input-8-548325381.py:8: DeprecationWarning: add_magnet_uri() is deprecated
  lt.add_magnet_uri(ses, magnet_link, params)


Enter Magnet Link Or Type Exit: exit


### Start Download
Source: https://stackoverflow.com/a/5494823/7957705 and [#3 issue](https://github.com/FKLC/Torrent-To-Google-Drive-Downloader/issues/3) which refers to this [stackoverflow question](https://stackoverflow.com/a/6053350/7957705)

In [9]:
import time
from IPython.display import display
import ipywidgets as widgets

state_str = [
    "queued",
    "checking",
    "downloading metadata",
    "downloading",
    "finished",
    "seeding",
    "allocating",
    "checking fastresume",
]

layout = widgets.Layout(width="auto")
style = {"description_width": "initial"}
download_bars = [
    widgets.FloatSlider(
        step=0.01, disabled=True, layout=layout, style=style
    )
    for _ in downloads
]
display(*download_bars)

while downloads:
    next_shift = 0
    for index, download in enumerate(downloads[:]):
        bar = download_bars[index + next_shift]
        if not download.is_seed():
            s = download.status()

            bar.description = " ".join(
                [
                    download.name(),
                    str(s.download_rate / 1000),
                    "kB/s",
                    state_str[s.state],
                ]
            )
            bar.value = s.progress * 100
        else:
            next_shift -= 1
            ses.remove_torrent(download)
            downloads.remove(download)
            bar.close() # Seems to be not working in Colab (see https://github.com/googlecolab/colabtools/issues/726#issue-486731758)
            download_bars.remove(bar)
            print(download.name(), "complete")
    time.sleep(1)


FloatSlider(value=0.0, disabled=True, layout=Layout(width='auto'), step=0.01, style=SliderStyle(description_wi…

Streaming output truncated to the last 5000 lines.
/tmp/ipython-input-9-3737971044.py:30: DeprecationWarning: is_seed() is deprecated
  if not download.is_seed():
/tmp/ipython-input-9-3737971044.py:35: DeprecationWarning: name() is deprecated
  download.name(),
/tmp/ipython-input-9-3737971044.py:30: DeprecationWarning: is_seed() is deprecated
  if not download.is_seed():
/tmp/ipython-input-9-3737971044.py:35: DeprecationWarning: name() is deprecated
  download.name(),
/tmp/ipython-input-9-3737971044.py:30: DeprecationWarning: is_seed() is deprecated
  if not download.is_seed():
/tmp/ipython-input-9-3737971044.py:35: DeprecationWarning: name() is deprecated
  download.name(),
/tmp/ipython-input-9-3737971044.py:30: DeprecationWarning: is_seed() is deprecated
  if not download.is_seed():
/tmp/ipython-input-9-3737971044.py:35: DeprecationWarning: name() is deprecated
  download.name(),
/tmp/ipython-input-9-3737971044.py:30: DeprecationWarning: is_seed() is deprecated
  if not download.is_s

www.1TamilMV.boo - Kerala Crime Files (2025) S02 EP(01-06) WEB-DL - 1080p - AVC - [Tam + Mal + Tel + Hin + Kan] - 5.8GB - ESub complete
